# find all domains in pe/ppe proteins

In [1]:
import sys,os,shutil,subprocess
import random
import glob, time
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from snpgenie import tools, app, trees
from prody import *

In [2]:
g = tools.genbank_to_dataframe(app.mbovis_gb)
g = g[g.feat_type!='gene']
g['gene']=g.gene.fillna('')
g=g[1:]
print (len(g))
x=g[(g.gene.str.contains('PE_PGRS') | g.gene.str.contains('PPE')) & (g.translation.str.len()>10)]
#for i,r in x.iterrows():
    #print ('LT708304.1','\t', r.start, '\t', r.end, r.gene, r.locus_tag)    

4199


In [3]:
seqs = [SeqRecord(seq=str(r.translation),id=str(r.locus_tag)) for i,r in x.iterrows()]
len(seqs)

133

In [6]:
def find_pfam_domains(g, res=None):
    """Find pfam domains"""
    
    new=[]
    for i,r in g.iterrows():
        if res!=None:
            if r.locus_tag in list(res.locus_tag):
                continue
        print (r.locus_tag)
        try:
            d = searchPfam(r.translation)
        except Exception as e:
            print (e)
            continue
        found = []
        for k in d:
            found = d[k]       
            found['locus_tag'] = r.locus_tag
            found['length'] = len(r.translation)
            found['product'] = r['product']
            new.append(found)    
    df=pd.DataFrame(new) 
    if len(df)>0 and g is not None:
        locs = df['locations'].apply(pd.Series)
        df=pd.concat([df, locs], axis=1)
        df=df.drop(columns=['locations'])
    df = pd.concat([res,df])
    return df

In [ ]:
#known = pd.read_csv('pfam_ppe_pe.csv',index_col=0)
known=None
res = find_pfam_domains(x[:15], known)
res['perc_domain'] = 
res.to_csv('pfam_ppe_pe.csv')

In [65]:
res

,accession,class,id,type,locus_tag,length,ali_end,ali_start,bitscore,end,cond_evalue,ind_evalue,evidence,hmm_end,hmm_start,start
0,PF00823.19,Domain,PPE,Pfam-A,BQ2027_MB0099,463,158,2,198.17,159,4.7e-63,8.5e-59,hmmer v3.0,157,1,2
1,PF00934.20,Domain,PE,Pfam-A,BQ2027_MB0113,496,93,1,108.77,93,7.2e-36,1.3e-31,hmmer v3.0,93,1,1
2,PF00934.20,Domain,PE,Pfam-A,BQ2027_MB0129,538,92,1,127.67,93,9.1e-42,1.6e-37,hmmer v3.0,92,1,1
3,PF00823.19,Domain,PPE,Pfam-A,BQ2027_MB0262c,556,164,8,191.49,165,5.4e-61,9.7e-57,hmmer v3.0,157,1,8
4,PF00934.20,Domain,PE,Pfam-A,BQ2027_MB0285c,868,93,1,114.74,93,9.8e-38,1.8e-33,hmmer v3.0,93,1,1
5,PF00934.20,Domain,PE,Pfam-A,BQ2027_MB0286c,877,93,1,120.17,93,2.0e-39,3.6e-35,hmmer v3.0,93,1,1
6,PF00934.20,Domain,PE,Pfam-A,BQ2027_MB0287c,831,93,1,116.88,93,2.1e-38,3.8e-34,hmmer v3.0,93,1,1
7,PF00823.19,Domain,PPE,Pfam-A,BQ2027_MB0288,536,162,6,197.66,163,6.8e-63,1.2e-58,hmmer v3.0,157,1,6
8,PF00823.19,Domain,PPE,Pfam-A,BQ2027_MB0294,513,164,8,205.34,165,3.0e-65,5.3e-61,hmmer v3.0,157,1,8
9,PF00934.20,Domain,PE,Pfam-A,BQ2027_MB0305,606,93,1,109.97,93,3.0e-36,5.4e-32,hmmer v3.0,93,1,1


In [7]:
labels=['hypothetical protein','conserved protein','conserved hypothetical',
        'unknown protein','uncharacterized','hypothetical alanine']
def find_unknown(x):
    for l in labels:
        if l in str(x).lower():           
            return True
    return False

g['hypothetical'] = g['product'].apply(find_unknown,1)
h = g[g.hypothetical==True]
print (len(h))


467


In [ ]:
#known = pd.read_csv('hypothetical_pfam.csv')
res2 = find_pfam_domains(h)
res2.to_csv('hypothetical_pfam.csv')

In [9]:
res2

,accession,class,id,type,locus_tag,length,product,ali_end,ali_start,bitscore,end,cond_evalue,ind_evalue,evidence,hmm_end,hmm_start,start
0,PF10774.9,Domain,DUF4226,Pfam-A,BQ2027_MB0026,120,CONSERVED HYPOTHETICAL PROTEIN,115,4,130.34,115,1.9e-42,3.4e-38,hmmer v3.0,111,1,4
1,PF10774.9,Domain,DUF4226,Pfam-A,BQ2027_MB0027,477,CONSERVED HYPOTHETICAL PROTEIN,184,75,122.33,184,5.8e-40,1.0e-35,hmmer v3.0,111,1,75
2,PF10824.8,Domain,T7SS_ESX_EspC,Pfam-A,BQ2027_MB0028,105,conserved hypothetical protein,98,1,100.28,99,3.8e-33,6.9e-29,hmmer v3.0,99,1,1
3,PF10904.8,Domain,DUF2694,Pfam-A,BQ2027_MB0029,101,conserved hypothetical protein,97,1,203.16,97,2.1e-65,3.8e-61,hmmer v3.0,97,1,1
4,PF10921.8,Domain,DUF2710,Pfam-A,BQ2027_MB0031,109,conserved hypothetical protein,106,3,211.77,106,7.9e-68,1.4e-63,hmmer v3.0,104,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,PF08310.11,Domain,LGFP,Pfam-A,BQ2027_MB3841,539,CONSERVED HYPOTHETICAL PROTEIN,471,420,58.03,471,1.1e-19,9.9e-16,hmmer v3.0,53,1,420
216,PF01510.25,Domain,Amidase_2,Pfam-A,BQ2027_MB3841,539,CONSERVED HYPOTHETICAL PROTEIN,373,221,55.70,374,7.3e-19,6.5e-15,hmmer v3.0,125,5,217
217,PF11196.8,Domain,DUF2834,Pfam-A,BQ2027_MB3861,160,HYPOTHETICAL PROTEIN,147,41,102.25,147,7.9e-34,1.4e-29,hmmer v3.0,96,1,41
218,PF06262.11,Domain,Zincin_1,Pfam-A,BQ2027_MB3866,116,CONSERVED HYPOTHETICAL PROTEIN,115,29,103.88,115,2.1e-34,3.7e-30,hmmer v3.0,96,1,29
